<p align="center"
    <a href="https://colab.research.google.com/github/ContextLab/psyc32-hello-world/blob/master/hello_world_assignment.ipynb" target="_parent">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab">
    </a>
</p>

In [1]:
import ast
import contextlib
import functools
import os
import signal
import subprocess
import sys
import types
from pathlib import Path

import pkg_resources
from IPython.core.ultratb import AutoFormattedTB

# **Test helpers**

In [2]:
setup_funcs = []
tests = []

In [3]:
class DavosTestingError:
    """Base class for Davos testing-related errors"""
    pass


class DavosAssertionError(DavosTestingError, AssertionError):
    """Subclasses AssertionError for pytest-specific handling"""
    pass


class TestingEnvironmentError(DavosAssertionError, OSError):
    """Raised due to issues with the testing environment"""
    pass


class NonColabEnvironmentError(TestingEnvironmentError):
    """Raised if this notebook is run outside of Colaboratory"""
    def __init__(self, *args):
        msg = "Tests in this notebook are meant to be run in a Colaboratory environment"
        super().__init__(msg, *args)


class TestTimeoutError(DavosAssertionError):
    """Raised if a test exceeds its timeout limit"""
    pass

In [4]:
def _expected_onion_parser_output(args_str, **installer_kwargs):
    _installer_kwargs = {'editable': False, 'spec': args_str}
    _installer_kwargs.update(installer_kwargs)
    return '"pip"', f'"""{args_str}"""', _installer_kwargs

In [5]:
def _expected_parser_output(name, as_=None, args_str=None, **installer_kwargs):
    if as_ is not None:
        as_ = f'"{as_}"'
    expected = f'smuggle(name="{name}", as_={as_}'
    if args_str is not None or any(installer_kwargs):
        installer, args_str, inst_kwargs = _expected_onion_parser_output(args_str, **installer_kwargs)
        expected += f', installer={installer}, args_str={args_str}, installer_kwargs={inst_kwargs}'
    return expected + ')'

In [6]:
def _is_imported(package):
    if package in sys.modules:
        return True
    elif any(pkg.startswith(f'{package}.') for pkg in sys.modules):
        return True
    return False

In [7]:
def _is_installed(package):
    try:
        dist = pkg_resources.get_distribution(package)
    except pkg_resources.DistributionNotFound:
        return False
    else:
        return True

In [8]:
def _matches_expected_output(expected, result):
    all_match = True
    expected_chunks = expected.split(';')
    result_chunks = result.split(';')
    assert len(expected_chunks) == len(result_chunks)
    for expected_chunk, result_chunk in zip(expected_chunks, result_chunks):
        if 'installer_kwargs=' in expected_chunk and 'installer_kwargs' in result_chunk:
            exp_main, exp_kwargs = expected_chunk.strip().split('installer_kwargs=')
            res_main, res_kwargs = result_chunk.strip().split('installer_kwargs=')
            # remove ')'
            exp_kwargs = ast.literal_eval(exp_kwargs[:-1])
            res_kwargs = ast.literal_eval(res_kwargs[:-1])
            all_match = all_match and exp_main == res_main and exp_kwargs == res_kwargs
        else:
            all_match = all_match and expected_chunk == result_chunk
    return all_match

In [9]:
def _parse_line(line):
    return davos.colab.smuggle_parser_colab(line)

In [10]:
def _parse_onion(onion):
    return davos.core.Onion.parse_onion(onion)

In [11]:
def setup_func(func):
    @functools.wraps(func)
    def _wrapped():
        return func()
        
    setup_funcs.append(func)
    return _wrapped

In [12]:
def test(func=None, *, timeout=100):
    def decorator(func):
        def handler(signum, frame):
            raise TestTimeoutError(
                f"'{func.__name__}' timed out after {timeout} seconds"
            )
        @functools.wraps(func)
        def wrapper():
            # print(f'timeout is {timeout}')
            signal.signal(signal.SIGALRM, handler)
            signal.alarm(timeout)
            try:
                return func()
            finally:
                signal.alarm(0)

        tests.append(wrapper)
        return wrapper
    
    if func is not None:
        return decorator(func)
    return decorator

In [13]:
def run_setup(source='github', ref=None, fork=None):
    n_funcs = len(setup_funcs)
    for i, sf in enumerate(setup_funcs):
        name = sf.__name__
        prefix = f"{i+1}/{n_funcs} {name}:"
        print(f"{prefix}{' '*(60-len(prefix))}", end='')
        try:
            if name == 'test_install_davos':
                sf(source=source, ref=ref, fork=fork)
            else:
                sf()
        except:
            print('failed')
            raise
        else:
            print('passed')

In [14]:
def show_failures(names_excs):
    tb_displayer = AutoFormattedTB('Context', 'LightBG')
    for name, exc in names_excs:
        n_pre = (85 - len(name) - 2) // 2
        n_post = 85 - len(name) - 2 - n_pre
        print(f"{'='*n_pre} {name} {'='*n_post}")
        print(tb_displayer.text(type(exc), exc, exc.__traceback__))
        print('\n\n\n')

In [15]:
def run_tests(live_stdout=False):
    if live_stdout:
        stdout_context = contextlib.nullcontext
    else:
        stdout_context = contextlib.redirect_stdout
    names_excs = []
    n_tests = len(tests)
    for i, tf in enumerate(tests, start=1):
        name = tf.__name__
        prefix = f"{i}/{n_tests}{' ' if i > 9 else '  '}{name}:"
        print(f"{prefix}{' '*(85-len(prefix))}", end='')
        try:
            with open(os.devnull, 'w') as devnull, stdout_context(devnull):
                tf()
        except Exception as e:
            names_excs.append((name, e))
            if isinstance(e, TestTimeoutError):
                print('timed out')
            else:
                print('failed')
        else:
            print('passed')
    
    print('\n\n')
    n_failed = len(names_excs)
    if n_failed == 0:
        print('All tests passed!')
    else:
        print(f'{n_tests-n_failed}/{n_tests} passed')
        print('Displaying errors from failing tests:')
        print('\n\n')
    show_failures(names_excs)

# **Setup tests**

### Testing environment validation & installation tests

In [16]:
@setup_func
def test_import_ipython():
    global IPython
    try:
        import IPython
    except ModuleNotFoundError as e:
        raise NonColabEnvironmentError() from e

In [17]:
@setup_func
def test_import_google():
    global google
    try:
        import google
    except ModuleNotFoundError as e:
        raise NonColabEnvironmentError() from e

In [18]:
@setup_func
def test_get_ipython_shell():
    global IPYTHON_SHELL
    try:
        IPYTHON_SHELL = get_ipython()
    except NameError as e:
        raise NonColabEnvironmentError() from e

In [19]:
@setup_func
def test_ipython_shell_is_colab_shell():
    try:
        assert isinstance(IPYTHON_SHELL, google.colab._shell.Shell)
    except AssertionError as e:
        raise NonColabEnvironmentError() from e
    except AttributeError as e:
        google_version = pkg_resources.get_distribution('google').version
        raise TestingEnvironmentError(
            "Qualified name for Colab shell class has changed (google module "
            "may have been recently updated).\n\tShell type:\t"
            f"{type(IPYTHON_SHELL)}\n\tgoogle version:\t{google_version}"
        ) from e

In [20]:
@setup_func
def test_ipython_version():
    ipy_version = IPython.__version__
    try:
        assert ipy_version == '5.5.0'
    except AssertionError as e:
        raise TestingEnvironmentError(
            f"Found unexpected IPython version: {ipy_version}\nExpected "
            "IPython==5.5.0"
        ) from e

In [21]:
@setup_func
def test_no_transforms_registered():
    assert len(IPYTHON_SHELL.input_splitter.python_line_transforms) == 0
    assert len(IPYTHON_SHELL.input_transformer_manager.python_line_transforms) == 0

In [22]:
@setup_func
def test_install_davos(source='github', ref=None, fork=None):
    source = source.lower()
    if source == 'github':
        if fork is None:
            fork = 'ContextLab'
        name = f'git+https://github.com/{fork}/davos.git'
        if ref is not None:
            name += f'@{ref}'
        name += '#egg=davos'
    elif source in ('pip', 'pypi', 'testpypi'):
        name = 'davos'
        if source == 'testpypi':
            name = '--index-url https://test.pypi.org/simple/ ' + name
        if ref is not None:
            name += f'=={ref}'
    elif source == 'conda':
        raise NotImplementedError(
            "conda installation is not supported in Colaboratory"
        )
    else:
        raise ValueError(f"Invalid source '{source}'")
    
    return_code = IPython.core.interactiveshell.system(f'pip install {name}')
    if return_code != 0:
        raise DavosTestingError(
            f"Failed to install 'davos'. Ran command:\n\t`pip install {name}`"
            )

### Import & initialization tests

In [23]:
@setup_func
def test_import_davos():
    global davos
    try:
        import davos
    except ModuleNotFoundError as e:
        raise DavosTestingError("Failed to import 'davos' after installation")

In [24]:
@setup_func
def test_smuggler_transform_registered():
    splitter_transforms = IPYTHON_SHELL.input_splitter.python_line_transforms
    line_transforms = IPYTHON_SHELL.input_transformer_manager.python_line_transforms
    # transform be registered once (and only once) in both places
    assert len(splitter_transforms) == 1
    assert len(line_transforms) == 1
    smuggler_splitter_transform = splitter_transforms[0]
    smuggler_line_transform = line_transforms[0]
    # both should be StatelessInputTransformers that wrap the 
    # `smuggle_parser_colab` function
    assert isinstance(smuggler_splitter_transform, 
                      IPython.core.inputtransformer.StatelessInputTransformer)
    assert isinstance(smuggler_line_transform, 
                      IPython.core.inputtransformer.StatelessInputTransformer)
    assert (smuggler_splitter_transform.func 
            is smuggler_line_transform.func 
            is davos.colab.smuggle_parser_colab)

In [25]:
@setup_func
def test_smuggle_function_in_namespace():
    assert 'smuggle' in IPYTHON_SHELL.user_ns

In [26]:
@setup_func
def test_correct_functions_chosen():
    assert smuggle is davos.colab.smuggle_colab

In [27]:
@setup_func
def test_Davos_object_initialization():
    davos_obj = davos.davos
    assert davos_obj.confirm_install is False
    assert davos_obj.suppress_stdout is False
    assert davos_obj.smuggled == {}
    assert davos_obj.smuggler is davos.colab.smuggle_colab
    assert davos.activate is davos_obj.activate_parser is davos.colab.activate_parser_colab
    assert davos.deactivate is davos_obj.deactivate_parser is davos.colab.deactivate_parser_colab
    assert davos_obj._shell_cmd_helper is davos.colab.run_shell_command_colab
    assert davos_obj.parser_environment == 'IPY_OLD'

# **Run setup tests**

In [28]:
run_setup(source='github', fork='paxtonfitzpatrick')

1/12 test_import_ipython:                                   passed
2/12 test_import_google:                                    passed
3/12 test_get_ipython_shell:                                passed
4/12 test_ipython_shell_is_colab_shell:                     passed
5/12 test_ipython_version:                                  passed
6/12 test_no_transforms_registered:                         passed
7/12 test_install_davos:                                    Collecting davos
  Cloning https://github.com/paxtonfitzpatrick/davos.git to /tmp/pip-install-sgjxfs1o/davos
  Running command git clone -q https://github.com/paxtonfitzpatrick/davos.git /tmp/pip-install-sgjxfs1o/davos
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for davos: filename=davos-0.0.1-cp37-none-any.whl size=34545 sha256=847e52e1511283ba9deb9a24ca4a34c33d05cbfa4e0637509782881cb5161422
  Stored in directory: /tmp/pip-ephem-wheel-

# **Main tests**

## Unit tests

### shell command runner tests

In [29]:
@test
def test_run_shell_command_simple():
    stdout, retcode = davos.davos.run_shell_command('whoami', live_stdout=False)
    assert retcode == 0
    assert stdout == 'root\r\n'

In [30]:
@test
def test_run_shell_command_multiword():
    quote = ("Strictly speaking, I didn't do the theiving. That would be the "
             "pirates. I just moved what they stole from one place to another")
    stdout, retcode = davos.davos.run_shell_command(f'echo "{quote}"', 
                                                    live_stdout=False)
    assert retcode == 0
    assert stdout == quote + '\r\n'

In [31]:
@test
def test_run_shell_command_failure():
    try:
        davos.davos.run_shell_command('blahblahblah', live_stdout=True)
    except subprocess.CalledProcessError as e:
        assert e.returncode == 127
        assert e.output == '/bin/sh: 1: blahblahblah: not found\r\n', e.output.getvalue()

### smuggle command parser tests

In [32]:
@test
def test_parser_ignores_line_no_smuggle():
    line = "def foo(bar, baz=qux):"
    assert _parse_line(line) == line

In [33]:
@test
def test_parser_ignores_line_decoy_smuggle():
    """a line that has "smuggle" in it, but should be ignored"""
    line = "def smuggle_something(foo):"
    assert _parse_line(line) == line

In [34]:
@test
def test_parser_ignores_commented_line():
    """parser should ignore commented-out smuggle statements"""
    line = "# smuggle foo as bar"
    assert _parse_line(line) == line

In [35]:
@test
def test_parser_handles_basic_line():
    """simplest use case"""
    line = "smuggle foo"
    expected = _expected_parser_output('foo')
    assert _matches_expected_output(expected, _parse_line(line))

In [36]:
@test
def test_parser_handles_basic_line_alias():
    """simplest use case, plus alias"""
    line = "smuggle foo as bar"
    expected = _expected_parser_output('foo', as_='bar')
    assert _matches_expected_output(expected, _parse_line(line))

In [37]:
@test
def test_parser_handles_basic_line_onion():
    """simplest use case, with onion comment"""
    line = "smuggle foo as bar    # pip: foo==0.0.1"
    expected = _expected_parser_output('foo', as_='bar', args_str='foo==0.0.1')
    assert _matches_expected_output(expected, _parse_line(line))

In [38]:
@test
def test_parser_ignores_non_onion_comment():
    """looks similar to the preivous one, but the inline comment is NOT an onion spec"""
    line = "smuggle foo as bar    # omg foo is my FAVORITE package"
    expected = _expected_parser_output('foo', as_='bar') + "    # omg foo is my FAVORITE package"
    assert _matches_expected_output(expected, _parse_line(line))

In [39]:
@test
def test_parser_ignores_imposter_onion_comment():
    """VERY close to an onion comment, but doesn't start with "pip:" (note colon)"""
    line = "smuggle foo as bar    # pip is a [redacted] package manager than conda"
    expected = _expected_parser_output('foo', as_='bar') + "    # pip is a [redacted] package manager than conda"
    assert _matches_expected_output(expected, _parse_line(line))

In [40]:
@test
def test_parser_handles_smuggle_qualname():
    line = "smuggle foo.bar as baz"
    expected = _expected_parser_output('foo.bar', as_='baz')
    assert _matches_expected_output(expected, _parse_line(line))

In [41]:
@test
def test_parser_handles_indented_line():
    """
    needs to handle, e.g.
        ```
        def foo():
            smuggle numpy as np
        ```
    Note: unless `davos.deactivate() is run, the above line will 
    actually be parsed (though it will have no effect)
    """
    line = "    smuggle foo as bar"
    expected = "    " + _expected_parser_output('foo', as_='bar')
    assert _matches_expected_output(expected, _parse_line(line))

In [42]:
@test
def test_parser_handles_different_install_name():
    """
    sometimes, name used for pip-installation is different 
    from name used to import
    """
    line = "smuggle foo as bar    # pip: foo-package==0.0.1"
    expected = _expected_parser_output('foo', as_='bar', 
                                       args_str='foo-package==0.0.1')
    assert _matches_expected_output(expected, _parse_line(line))

In [43]:
@test
def test_parser_handles_multiple_packages():
    line = "smuggle foo as bar, baz as qux, spam, ham, eggs"
    expected = _expected_parser_output('foo', as_='bar')
    expected += f"; {_expected_parser_output('baz', as_='qux')}"
    expected += f"; {_expected_parser_output('spam')}"
    expected += f"; {_expected_parser_output('ham')}"
    expected += f"; {_expected_parser_output('eggs')}"
    assert _matches_expected_output(expected, _parse_line(line))

In [44]:
@test
def test_parser_handles_multiple_packages_onion():
    """onion info should be passed to smuggle function for FIRST package"""
    line = "smuggle foo as bar, baz as qux, spam, ham, eggs    # pip: foo==0.0.1"
    expected = _expected_parser_output('foo', as_='bar', args_str='foo==0.0.1')
    expected += f"; {_expected_parser_output('baz', as_='qux')}"
    expected += f"; {_expected_parser_output('spam')}"
    expected += f"; {_expected_parser_output('ham')}"
    expected += f"; {_expected_parser_output('eggs')}"
    assert _matches_expected_output(expected, _parse_line(line))

In [45]:
@test
def test_parser_handles_backslash():
    line = """smuggle foo as bar, \
                      baz as qux, \
                      spam, ham, \
                      eggs"""
    expected = _expected_parser_output('foo', as_='bar')
    expected += f"; {_expected_parser_output('baz', as_='qux')}"
    expected += f"; {_expected_parser_output('spam')}"
    expected += f"; {_expected_parser_output('ham')}"
    expected += f"; {_expected_parser_output('eggs')}"
    assert _matches_expected_output(expected, _parse_line(line))

In [46]:
@test
def test_parser_handles_backslash_onion():
    """again, onion info should be passed to smuggle function for FIRST package"""
    line = """smuggle foo as bar, \
                      baz as qux, \
                      spam, ham, \
                      eggs    # pip: foo==0.0.1"""
    expected = _expected_parser_output('foo', as_='bar', args_str='foo==0.0.1')
    expected += f"; {_expected_parser_output('baz', as_='qux')}"
    expected += f"; {_expected_parser_output('spam')}"
    expected += f"; {_expected_parser_output('ham')}"
    expected += f"; {_expected_parser_output('eggs')}"
    assert _matches_expected_output(expected, _parse_line(line))

In [47]:
@test
def test_parser_handles_inconsistent_whitespace():
    """should handle weird amounts of whitespace that are *technically* valid"""
    line = """smuggle               foo     as    bar    \
,baz as qux  , \
          spam  .  ham    as    eggs                #   pip   :       foo==0.0.1    """
    expected = _expected_parser_output('foo', as_='bar', args_str='foo==0.0.1')
    expected += f"; {_expected_parser_output('baz', as_='qux')}"
    expected += f"; {_expected_parser_output('spam.ham', as_='eggs')}"
    # real parser adds back leading & trailing characters (including whitespace)
    expected += "    "
    assert _matches_expected_output(expected, _parse_line(line))

In [48]:
@test
def test_parser_handles_smuggle_from():
    """second possible broad syntax class"""
    line = "from foo smuggle bar"
    expected = _expected_parser_output('foo.bar', as_='bar')
    assert _matches_expected_output(expected, _parse_line(line))

In [49]:
@test
def test_parser_handles_smuggle_from_onion():
    """second possible broad syntax class"""
    line = "from foo smuggle bar    # pip: foo==0.0.1"
    expected = _expected_parser_output('foo.bar', as_='bar', args_str='foo==0.0.1')
    assert _matches_expected_output(expected, _parse_line(line))

In [50]:
@test
def test_parser_handles_smuggle_from_multi():
    line = "from foo smuggle bar, baz as spam, qux"
    expected = _expected_parser_output('foo.bar', as_='bar')
    expected += f"; {_expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {_expected_parser_output('foo.qux', as_='qux')}"
    assert _matches_expected_output(expected, _parse_line(line))

In [51]:
@test
def test_parser_handles_smuggle_from_multi_onion():
    """onion info should be passed to FIRST smuggle function"""
    line = "from foo smuggle bar, baz as spam, qux    # pip: foo==0.0.1"
    expected = _expected_parser_output('foo.bar', as_='bar', args_str='foo==0.0.1')
    expected += f"; {_expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {_expected_parser_output('foo.qux', as_='qux')}"
    assert _matches_expected_output(expected, _parse_line(line))

In [52]:
@test
def test_parser_handles_smuggle_from_backslash():
    """onion info should be passed to FIRST smuggle function"""
    line = """from foo smuggle bar, \
                               baz as spam, \
                               qux    # pip: foo==0.0.1"""
    expected = _expected_parser_output('foo.bar', as_='bar', args_str='foo==0.0.1')
    expected += f"; {_expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {_expected_parser_output('foo.qux', as_='qux')}"
    assert _matches_expected_output(expected, _parse_line(line))

In [53]:
@test
def test_parser_handles_smuggle_from_parentheses():
    line = """from foo smuggle (bar, baz as spam, qux,)"""
    # also tests trailing comma inside parentheses, which is valid
    expected = _expected_parser_output('foo.bar', as_='bar')
    expected += f"; {_expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {_expected_parser_output('foo.qux', as_='qux')}"
    assert _matches_expected_output(expected, _parse_line(line))

In [54]:
@test
def test_parser_handles_smuggle_from_parentheses_multiline_1():
    line = """from foo smuggle (bar, 
                                baz as spam, 
                                qux)"""
    expected = _expected_parser_output('foo.bar', as_='bar')
    expected += f"; {_expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {_expected_parser_output('foo.qux', as_='qux')}"
    assert _matches_expected_output(expected, _parse_line(line))

In [55]:
@test
def test_parser_handles_smuggle_from_parentheses_multiline_1_onion_1():
    line = """from foo smuggle (bar,    # pip: foo==0.0.1
                                baz as spam, 
                                qux)"""
    expected = _expected_parser_output('foo.bar', as_='bar', args_str='foo==0.0.1')
    expected += f"; {_expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {_expected_parser_output('foo.qux', as_='qux')}"
    assert _matches_expected_output(expected, _parse_line(line))

In [56]:
@test
def test_parser_handles_smuggle_from_parentheses_multiline_1_onion_2():
    line = """from foo smuggle (bar, 
                                baz as spam, 
                                qux)    # pip: foo==0.0.1"""
    expected = _expected_parser_output('foo.bar', as_='bar', args_str='foo==0.0.1')
    expected += f"; {_expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {_expected_parser_output('foo.qux', as_='qux')}"
    assert _matches_expected_output(expected, _parse_line(line))

In [57]:
@test
def test_parser_handles_smuggle_from_parentheses_multiline_2():
    line = """from foo smuggle (
                  bar, 
                  baz as spam, 
                  qux
              )"""
    expected = _expected_parser_output('foo.bar', as_='bar')
    expected += f"; {_expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {_expected_parser_output('foo.qux', as_='qux')}"
    assert _matches_expected_output(expected, _parse_line(line))

In [58]:
@test
def test_parser_handles_smuggle_from_parentheses_multiline_2_onion_1():
    line = """from foo smuggle (    # pip: foo==0.0.1
                  bar, 
                  baz as spam, 
                  qux
              )"""
    expected = _expected_parser_output('foo.bar', as_='bar', args_str='foo==0.0.1')
    expected += f"; {_expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {_expected_parser_output('foo.qux', as_='qux')}"
    assert _matches_expected_output(expected, _parse_line(line))

In [59]:
@test
def test_parser_handles_smuggle_from_parentheses_multiline_2_onion_2():
    line = """from foo smuggle (
                  bar, 
                  baz as spam, 
                  qux
              )    # pip: foo==0.0.1"""
    expected = _expected_parser_output('foo.bar', as_='bar', args_str='foo==0.0.1')
    expected += f"; {_expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {_expected_parser_output('foo.qux', as_='qux')}"
    assert _matches_expected_output(expected, _parse_line(line))

In [60]:
@test
def test_parser_handles_smuggle_from_parentheses_multiline_2_onion_2_comments():
    line = """from foo smuggle (    # unrelated comment on first line
                  bar,    # unrelated comment on package name line
                  baz as spam, 
                  # unrelated comment on its own line
                  qux
              )    # pip: foo==0.0.1    # unrelated comment after onion"""
    expected = _expected_parser_output('foo.bar', as_='bar', args_str='foo==0.0.1')
    expected += f"; {_expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {_expected_parser_output('foo.qux', as_='qux')}"
    expected += "    # unrelated comment after onion"
    assert _matches_expected_output(expected, _parse_line(line))

In [61]:
@test
def test_parser_handles_smuggle_semicolons():
    """also combines multiple elements from prior tests"""
    line = """smuggle foo; smuggle bar as baz; \
              from spam smuggle eggs; \
              from qux smuggle quux as corge    # pip: qux-package==0.0.1"""
    expected = _expected_parser_output('foo')
    expected += f"; {_expected_parser_output('bar', as_='baz')}"
    expected += f"; {_expected_parser_output('spam.eggs', as_='eggs')}"
    expected += f"; {_expected_parser_output('qux.quux', as_='corge', args_str='qux-package==0.0.1')}"
    assert _matches_expected_output(expected, _parse_line(line))

### onion comment parser tests

In [62]:
@test
def test_onion_parser_simple():
    onion = '# pip: foo==0.0.1'
    expected = _expected_onion_parser_output('foo==0.0.1')
    assert _parse_onion(onion) == expected

In [63]:
@test
def test_onion_parser_whitespace():
    onion = '#             pip:            foo==0.0.1'
    expected = _expected_onion_parser_output('foo==0.0.1')
    assert _parse_onion(onion) == expected

In [64]:
@test
def test_onion_parser_github():
    onion = '# pip: git+https://github.com/foo/bar.git@branch-name#egg=foo&subdirectory=baz'
    expected = _expected_onion_parser_output('git+https://github.com/foo/bar.git@branch-name#egg=foo&subdirectory=baz')
    assert _parse_onion(onion) == expected

In [65]:
@test
def test_onion_parser_editable():
    onion = '# pip: --editable git+https://github.com/foo/bar.git'
    expected = _expected_onion_parser_output('--editable git+https://github.com/foo/bar.git', 
                                             editable=True, 
                                             spec='git+https://github.com/foo/bar.git')
    assert _parse_onion(onion) == expected

In [66]:
@test
def test_onion_parser_joined_short_args():
    onion = '# pip: -Ive git+https://github.com/foo/bar.git'
    expected = _expected_onion_parser_output('-Ive git+https://github.com/foo/bar.git', 
                                             editable=True, 
                                             ignore_installed=True, 
                                             verbosity=1,
                                             spec='git+https://github.com/foo/bar.git')
    assert _parse_onion(onion) == expected

In [67]:
@test
def test_onion_parser_fails_mutually_exclusive_args():
    onion = '# pip: --use-pep517 --no-use-pep517 foo'
    try:
        _parse_onion(onion)    # SHOULD FAIL
    except davos.exceptions.OnionArgumentError as e:
        # has both 'msg' and 'message' attrs because it inherits from both 
        # SyntaxError and argparse.ArgumentError
        assert e.msg == e.message == "not allowed with argument --use-pep517"
        assert e.argument_name == "--no-use-pep517"
        assert str(e) == "argument --no-use-pep517: not allowed with argument --use-pep517"
    else:
        raise DavosAssertionError("test should've raised 'davos.exceptions.OnionArgumentError'")

In [68]:
@test
def test_onion_parser_fails_editable_after_spec():
    onion = '# pip: git+https://github.com/foo/bar.git#egg=foo --editable'
    try:
        _parse_onion(onion)    # SHOULD FAIL
    except davos.exceptions.OnionArgumentError as e:
        # has both 'msg' and 'message' attrs because it inherits from both 
        # SyntaxError and argparse.ArgumentError
        assert e.msg == e.message == "expected one argument"
        assert e.argument_name == "-e/--editable"
        assert str(e) == "argument -e/--editable: expected one argument"
    else:
        raise DavosAssertionError("test should've raised 'davos.exceptions.OnionArgumentError'")

In [69]:
@test
def test_onion_parser_fails_arg_requires_value():
    onion = '# pip: foo==0.0.1 --platform '
    try:
        _parse_onion(onion)    # SHOULD FAIL
    except davos.exceptions.OnionArgumentError as e:
        # has both 'msg' and 'message' attrs because it inherits from both 
        # SyntaxError and argparse.ArgumentError
        assert e.msg == e.message == "expected one argument"
        assert e.argument_name == "--platform"
        assert str(e) == "argument --platform: expected one argument"
    else:
        raise DavosAssertionError("test should've raised 'davos.exceptions.OnionArgumentError'")

### miscellaneous tests

In [70]:
@test
def test_does_not_register_multiple_transformers():
    splitter_xforms = IPYTHON_SHELL.input_splitter.python_line_transforms
    manager_xforms = IPYTHON_SHELL.input_transformer_manager.python_line_transforms
    initial_n_splitter_xforms = len(splitter_xforms)
    initial_n_manager_xforms = len(manager_xforms)
    # record whether parser is active before running test to restore 
    # initial state afterward
    start_active = davos.is_active()
    if not start_active:
        # ensure these are the number of registered transforms 
        # *including the davos parser*
        initial_n_splitter_xforms += 1
        initial_n_manager_xforms += 1

    try:
        for _ in range(5):
            davos.activate()
            assert len(splitter_xforms) == initial_n_splitter_xforms
            assert len(manager_xforms) == initial_n_manager_xforms
    finally:
        if not start_active:
            davos.deactivate()

In [71]:
@test
def test_deactivate_reactivate():
    start_active = davos.is_active()
    try:
        # start with the parser active
        if not start_active:
            davos.activate()
        
        assert davos.is_active()
        davos.deactivate()
        assert not davos.is_active()
        davos.activate()
        assert davos.is_active()
    finally:
        # return to state before function
        if not start_active:
            davos.deactivate()

## Integration tests

In [72]:
@test(timeout=30)
def test_smuggle_pypi_new():
    assert not _is_installed('ppca')
    smuggle ppca    # pip: ppca>=0.0.4
    assert isinstance(ppca, types.ModuleType)
    assert hasattr(ppca, 'PPCA')

In [73]:
@test(timeout=30)
def test_smuggle_from_pypi_new():
    assert not _is_installed('umap-learn')
    from umap smuggle UMAP    # pip: umap-learn==0.4.6
    assert hasattr(UMAP, 'fit')
    assert _is_installed('umap-learn==0.4.6')

In [74]:
@test(timeout=30)
def test_smuggle_previously_installed():
    assert _is_installed('fastdtw==0.3.4')
    smuggle fastdtw    # pip: fastdtw==0.3.2
    assert _is_installed('fastdtw==0.3.2')

In [75]:
@test(timeout=30)
def test_smuggle_previously_imported():
    import tqdm
    assert tqdm.__version__ == '4.41.1'
    smuggle tqdm    # pip: tqdm==4.45.0
    assert tqdm.__version__ == '4.45.0'

In [76]:
@test(timeout=30)
def test_smuggle_github_ref():
    assert not _is_installed('hypertools')
    smuggle hypertools as hyp    # pip: git+https://github.com/ContextLab/hypertools.git@36c12fd#egg=hypertools
    assert isinstance(hyp, types.ModuleType)
    assert hyp is sys.modules['hypertools']
    assert hasattr(hyp, 'DataGeometry')
    assert hyp.__version__ == '0.6.3'
    assert _is_installed('hypertools==0.6.3')

In [77]:
@test(timeout=30)
def test_smuggle_github_editable():
    assert not _is_installed('quail')
    smuggle quail    # pip: -e git+https://github.com/ContextLab/quail.git@v0.2.0#egg=quail --src /content/gh_clones
    assert '/content/gh_clones/quail' in sys.path
    assert isinstance(quail, types.ModuleType)
    assert hasattr(quail, 'Egg')
    assert _is_installed('quail==0.2.0')
    assert Path('/content/gh_clones/quail').is_dir()

In [78]:
@test(timeout=300)
def test_smuggle_github_subdirectory():
    assert not _is_installed('sherlock_helpers')
    smuggle sherlock_helpers    # pip: git+https://github.com/ContextLab/sherlock-topic-model-paper.git@v1.0#subdirectory=code/sherlock_helpers
    assert isinstance(sherlock_helpers, types.ModuleType)
    assert hasattr(sherlock_helpers, 'github_url')
    assert sherlock_helpers.__version__ == '0.0.1'
    assert _is_installed('sherlock_helpers==0.0.1')

# **Run main tests**

In [79]:
run_tests()

1/50  test_run_shell_command_simple:                                                 passed
2/50  test_run_shell_command_multiword:                                              passed
3/50  test_run_shell_command_failure:                                                passed
4/50  test_parser_ignores_line_no_smuggle:                                           passed
5/50  test_parser_ignores_line_decoy_smuggle:                                        passed
6/50  test_parser_ignores_commented_line:                                            passed
7/50  test_parser_handles_basic_line:                                                passed
8/50  test_parser_handles_basic_line_alias:                                          passed
9/50  test_parser_handles_basic_line_onion:                                          passed
10/50 test_parser_ignores_non_onion_comment:                                         passed
11/50 test_parser_ignores_imposter_onion_comment:                               

Helper functions and variables used across multiple notebooks can be found in `/usr/local/lib/python3.7/dist-packages/sherlock_helpers`, or on GitHub, [here](https://github.com/ContextLab/sherlock-topic-model-paper/tree/master/code/sherlock_helpers).<br />You can also view source code directly from the notebook with:<br /><pre>    from sherlock_helpers.functions import show_source<br />    show_source(foo)</pre>

passed



All tests passed!
